In [1]:
import numpy as np
from uncertainties import ufloat
from uncertainties import unumpy

import astropy.io.fits as pyfits
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.time import Time
from astropy.wcs import WCS

from sys import argv
import warnings
import time
import os

In [2]:
path_log  = '../File_Log_imfit/'
path_data = '../Data_NoSelfcal/Data_NoSelfCal_Rg/'

outnames_log = [
                'iras4a2_Q_2014_10_13',
                'iras4a2_K_2016_06_04',
                'iras4a2_K_2016_08_06',
                'iras4a2_Ka_2016_09_04',
                'iras4a2_B7_2016_09_06',
                'iras4a2_B6_2016_11_04',
                'iras4a2_B3_2017_08_17',
                'iras4a2_B4_2019_07_20'
            ]

outnames_fits = [
                'n1333iras4a_Q_rob-2_uvAll_2014_10_13',
                'n1333iras4a_K_rob-2_uvAll_2016_06_04',
                'n1333iras4a_K_rob-2_uvAll_2016_08_06',
                'n1333iras4a_Ka_rob-2_uvAll_2016_09_04',
                'n1333iras4a_B7_rob-2_uvAll_2016_09_06',
                'n1333iras4a_B6_rob-2_uvAll_2016_11_04',
                'n1333iras4a_B3_rob-2_uvAll_2017_08_17',
                'n1333iras4a_B4_rob-2_uvAll_2019_07_20'
            ]

### Positions of NGC1333 IRAS4A2 from the 2D gaussian fitting

Hint: position inforamation is obtained from CASA imfit, run the following scripts first. 
- script_iras4a_rg2b4.py
- script_iras4a_imfit.py

In [3]:
# Obtain the coordinate information (Coord_4A2_arr)

ra_gaufit_hms_list, ra_err_gaufit_sec_list, ra_err_gaufit_arcs_list = [], [], []
dec_gaufit_dms_list, dec_err_gaufit_arcs_list = [], []
Coord_4A2_list = []

for i, outname_log in enumerate(outnames_log):

    # Set the log filename
    logName  = '%s%s.log'%(path_log, outname_log)

    # read the log file
    file  = open(logName,"r+") 
    lines = file.read().splitlines()
    file.close()
    
    # save the ra & dec
    ra_h        = lines[20].split()[2].split(':')[0]
    ra_m        = lines[20].split()[2].split(':')[1]
    ra_s        = lines[20].split()[2].split(':')[2]   
    ra_hms      = ra_h + 'h' + ra_m + 'm' + ra_s + 's'
    ra_err_sec  = lines[20].split()[4]
    ra_err_arcs = lines[20].split()[6].split('(')[1]
    
    dec_d       = lines[21].split()[2].split('.', 2)[0]
    dec_m       = lines[21].split()[2].split('.', 2)[1]
    dec_s       = lines[21].split()[2].split('.', 2)[2]
    dec_dms     = dec_d + 'd' + dec_m + 'm' + dec_s + 's'
    dec_err_arcs = lines[21].split()[4]
    
    Coord_4A2 = SkyCoord(ra_hms, dec_dms, frame='icrs')
    
    # add to the list
    ra_gaufit_hms_list.append(ra_hms)
    ra_err_gaufit_sec_list.append(ra_err_sec)
    ra_err_gaufit_arcs_list.append(ra_err_arcs)
    dec_gaufit_dms_list.append(dec_dms)
    dec_err_gaufit_arcs_list.append(dec_err_arcs)
    Coord_4A2_list.append(Coord_4A2) 
     
# convert list to array
ra_gaufit_hms_arr       = np.array(ra_gaufit_hms_list)
ra_err_gaufit_sec_arr   = np.array(ra_err_gaufit_sec_list).astype(np.float)
ra_err_gaufit_arcs_arr  = np.array(ra_err_gaufit_arcs_list).astype(np.float)
dec_gaufit_dms_arr      = np.array(dec_gaufit_dms_list)
dec_err_gaufit_arcs_arr = np.array(dec_err_gaufit_arcs_list).astype(np.float)
Coord_4A2_arr           = np.array(Coord_4A2_list)

In [4]:
# Coordinate of IRAS4A2 [RA:deg; Dec:deg]
# hints: Gaussian fitting via CASA imfit
print (Coord_4A2_arr, '\n')
print (ra_gaufit_hms_arr, '\n')
print (dec_gaufit_dms_arr, '\n')

[<SkyCoord (ICRS): (ra, dec) in deg
    (52.29345667, 31.22559842)>
 <SkyCoord (ICRS): (ra, dec) in deg
    (52.29344759, 31.22558165)>
 <SkyCoord (ICRS): (ra, dec) in deg
    (52.29345217, 31.22557831)>
 <SkyCoord (ICRS): (ra, dec) in deg
    (52.29345153, 31.22557847)>
 <SkyCoord (ICRS): (ra, dec) in deg
    (52.29345869, 31.22557215)>
 <SkyCoord (ICRS): (ra, dec) in deg
    (52.29345734, 31.22557471)>
 <SkyCoord (ICRS): (ra, dec) in deg
    (52.29345824, 31.225573)>
 <SkyCoord (ICRS): (ra, dec) in deg
    (52.29346182, 31.22556535)>] 

['03h29m10.42960s' '03h29m10.427422s' '03h29m10.428520s'
 '03h29m10.428368s' '03h29m10.430085s' '03h29m10.429762s'
 '03h29m10.429977s' '03h29m10.430836s'] 

['+031d13m32.15433s' '+031d13m32.093957s' '+031d13m32.081908s'
 '+031d13m32.082498s' '+031d13m32.059731s' '+031d13m32.068950s'
 '+031d13m32.062794s' '+031d13m32.035269s'] 



### Observation time from the header 

In [5]:
# obtain the time information (Time_arr)

Time_list = []

for i, outname_fits in enumerate(outnames_fits):
    
    # Get the FITS and header
    DataName  = '%s%s_I.image.fits'%(path_data, outname_fits)
    I_hd      = pyfits.getheader(DataName)
    
    # Get the frequency, beam, and observation time information from header
    hd_freq = float(I_hd['RESTFRQ'])
    hd_bmaj = float(I_hd['BMAJ'])
    hd_bmin = float(I_hd['BMIN'])
    hd_bpa  = float(I_hd['BPA'])
    hd_time = I_hd['DATE-OBS']
    
    Time_ymd = Time(hd_time.split('T')[0])
    Time_list.append(Time_ymd)
    
# convert list to array    
Time_arr = np.array(Time_list)

In [6]:
print (Time_arr)

[<Time object: scale='utc' format='iso' value=2014-10-13 00:00:00.000>
 <Time object: scale='utc' format='iso' value=2016-06-04 00:00:00.000>
 <Time object: scale='utc' format='iso' value=2016-08-06 00:00:00.000>
 <Time object: scale='utc' format='iso' value=2016-09-04 00:00:00.000>
 <Time object: scale='utc' format='iso' value=2016-09-06 00:00:00.000>
 <Time object: scale='utc' format='iso' value=2016-11-04 00:00:00.000>
 <Time object: scale='utc' format='iso' value=2017-08-17 00:00:00.000>
 <Time object: scale='utc' format='iso' value=2019-07-20 00:00:00.000>]


In [7]:
# Observational bands
Band_arr = np.array(['Q', 'K', 'K', 'Ka', 'B7', 'B6', 'B3', 'B4'])

# Uncertainties of coordinate towards IRAS4A2 [arcsec]
Coord_err_4A2_arr = \
np.array([39e-3,\
          25e-3,\
          24e-3,\
          20e-3,\
          50e-3,\
          19e-3,\
          13e-3,\
          3e-3,\
         ])

### Save the information to the textfile

In [8]:
%%time

txt_lists = []
for i, elm in enumerate(Band_arr):
    
    # Time
    Time_ymd       = str(Time_arr[i]).split()[0]                 # Observationl Time [y-m-d]
    Time_yr        = Time_arr[i].decimalyear                     # Observationl Time [year]
    Time_jd        = Time_arr[i].jd                              # Observationl Time [Julian Day]
    
    # Coordinate System
    radec_hmsdms   = Coord_4A2_arr[i].to_string('hmsdms')        # RA & Dec [hmsdms]
    radec_err_arcs = Coord_err_4A2_arr[i]                        # Uncertainties of RA & Dec [arcsec]
    radec_err_deg  = np.divide(Coord_err_4A2_arr[i], 3600)       # Uncertainties of RA & Dec [degree]
    radec_dig_deg  = int((np.floor(np.log10(radec_err_deg))*-1)) # Significant digit of RA & Dec
    
    ra_hms         = str(radec_hmsdms).split()[0]                # Right Ascension [hms]
    dec_dms        = str(radec_hmsdms).split()[1]                # Declination [dms]
    ra_deg         = Coord_4A2_arr[i].ra.deg                     # Right Ascension [degree]
    dec_deg        = Coord_4A2_arr[i].dec.deg                    # Declination [degree]
    ra_dig_deg     = round(ra_deg, radec_dig_deg+1)              # Right Ascension with significant digit [degree]
    dec_dig_deg    = round(dec_deg, radec_dig_deg+1)             # Declination with significant digit  [degree]
   
    # save the txt file   
    txt_list = [i+1, elm, Time_ymd, Time_yr, Time_jd, ra_hms, dec_dms, ra_dig_deg, dec_dig_deg, ra_deg, dec_deg, radec_err_deg, '\\\\']
    # print (txt_list)
    txt_lists.append(txt_list)
        
    np.savetxt('Position_4A2.txt', txt_lists, fmt='%s', delimiter=' & ')

CPU times: user 21.7 ms, sys: 6.54 ms, total: 28.3 ms
Wall time: 28.6 ms
